<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

- Lat: -90 South pole to +90 North Pole (equator=0)
- Long: Greenwich 0 (-180 to +180)

*note: EE takes care of the modulo + no need in Venezuela)*

In [21]:
import math
import time

In [1]:
import ee

# authenticating a GCloud service account (id, privatekey)
service_id = 'ee-access@goldbusters-data-load.iam.gserviceaccount.com'
service_privatekey = 'goldbusters-data-load-privatekey.json'
credentials = ee.ServiceAccountCredentials(service_id, service_privatekey)
ee.Initialize(credentials)

# WIP

When exporting an image, we cut a **region** from a **scene**:
- scene: defined by dataset, date, cloud coverage, bands
- region: polygon drawing the region of interest (defined by a clockwise list of [lat,lon])

### Get Scene

In [2]:
# dataset and bands
dataset = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2'])

# filter on dates, sort by cloud coverage (asc)
scenes = dataset.filter(ee.Filter.date('2017-04-01', '2017-09-02')).sort('CLOUD_COVER')

# Scene: pick the one with lowest cloud coverage (clearest)
scene = scenes.first()

### Get Region of Interest

In [16]:
# offset a point by a shift, convert shift from meters to a lat/long offset
def offset(point, shift_x, shift_y):
    #Earth’s radius, sphere
    R=6378137

    #Coordinate offsets in radians
    lat, lon = point
    dLat = shift_x/R
#     print(lat)
#     print(type(lat))
#     print(math.pi*lat)
    dLon = shift_y/(R*math.cos(math.pi*lat/180))

    return lat + dLat * 180/math.pi, lon + dLon * 180/math.pi

# get image from center point + size
def get_ROI(center, size):
    """ Get coords of a square Regoin Of Interest from a center point coords and specified size
    inputs:
        center (tuple): coords of the center point for the tile eg:(lat, long)
        size (int): shift (square region)
    output:
        ROI (list(list)): coords of bottom-left, upper-right corners of AOI
    """
    lower_left_x, lower_left_y = offset(center, -size/2, -size/2)
    upper_right_x, upper_right_y = offset(center, size/2, size/2)
    
    #region defined by its four corners (it's a polygon, clock-wise)
    ROI = [[lower_left_x, lower_left_y],
          [lower_left_x, upper_right_y],
          [upper_right_x, upper_right_y],
          [upper_right_x, lower_left_y]]
    
    return ee.Geometry.Polygon(ROI) 

### Cookie-cut the ROI from the Scene, export to GCS

In [10]:
# Google Cloud Storage bucket name
gcs_bucket = 'auven-satellite-data'

In [17]:
center = (-78.614217766, -0.827621582972643)
zoom_level = 17
size_ROI = 2000 # 2km by 2km

# get region from center + size
ROI = get_ROI(center, size_ROI)

*Note: the region argument passed to Export.image should be a **list of Polygons**.*

*Way to do this:*
- [ROI]
- ROI.getInfo()['coordinates']

In [27]:
[ROI]

[ee.Geometry({
   "functionInvocationValue": {
     "functionName": "GeometryConstructors.Polygon",
     "arguments": {
       "coordinates": {
         "constantValue": [
           [
             [
               -78.62320091884119,
               -0.8731256964561477
             ],
             [
               -78.62320091884119,
               -0.7821174694891383
             ],
             [
               -78.6052346131588,
               -0.7821174694891383
             ],
             [
               -78.6052346131588,
               -0.8731256964561477
             ]
           ]
         ]
       },
       "evenOdd": {
         "constantValue": true
       }
     }
   }
 })]

In [24]:
ROI.getInfo()['coordinates']

[[[-78.62320091884119, -0.8731256964561477],
  [-78.6052346131588, -0.8731256964561477],
  [-78.6052346131588, -0.7821174694891383],
  [-78.62320091884119, -0.7821174694891383],
  [-78.62320091884119, -0.8731256964561477]]]

In [33]:
config = {'scale': zoom_level,
          'region': ROI.getInfo()['coordinates'],
          'bucket': gcs_bucket}

# Export the image, specifying scene, region and scale (= zoom level).
task = ee.batch.Export.image.toCloudStorage(scene, 'testExportGeoTIFF', **config)
#     **{
#     'image': scene,
#     'description': 'testExportGeoTIFF',
#     'region': ROI.getInfo()['coordinates'],
#     'fileFormat': 'GeoTIFF',
#     'bucket': gcs_bucket
# })

# Send the task to the earth engine.
task.start()

In [34]:
# Monitor the task.
while task.status()['state'] in ['READY', 'RUNNING']:
    print(task.status())
    time.sleep(10)
else:
    print(task.status())

{'state': 'READY', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244066518, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'READY', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244066518, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'READY', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244066518, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'RUNNING', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timesta

{'state': 'RUNNING', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244307048, 'start_timestamp_ms': 1620244094215, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'RUNNING', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244307048, 'start_timestamp_ms': 1620244094215, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'RUNNING', 'description': 'testExportGeoTIFF', 'creation_timestamp_ms': 1620244066518, 'update_timestamp_ms': 1620244367061, 'start_timestamp_ms': 1620244094215, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'JHY3QY5R6IZ2LMH5PDVBDDPC', 'name': 'projects/earthengine-legacy/operations/JHY3QY5R6IZ2LMH5PDVBDDPC'}
{'state': 'RUNNING', 'descri

In [38]:
if task.status()['state'] == 'COMPLETED':
    print('#############\n   SUCCESS\n#############')
else:
    print('------------\n...fail...\n------------')

#############
   SUCCESS
#############


## Misc -- notes, archive

In [ ]:
# find images with some filters
ee.ui.Map.setCenter(-78.614217766, -0.827621582972643, 17)
temporal_snapshot = dataset.filter(ee.Filter.date('2017-04-01', '2017-09-02'))
sorted = temporal_snapshot.sort('CLOUD_COVER')
scene = sorted.first()

In [ ]:
task = ee.batch.Export.image.toCloudStorage(**{
    'image': scene,
    'description': 'firstImageTest',
    'scale': 100,
    'region': geometry.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'bucket': gcs_bucket,
    'formatOptions': {'cloudOptimized': True}